# Collecting Data with Panda

## Runing Panda
* Run block "Start Panda" from a termnial with the same path as this notebook
    * The reason for this is that you have to interact with the terminal after QEMU boots
* To start recording the session
    * `begin_record <recording_name>`
* To end the recording 
    * `end_record`

In [170]:
%%bash
. ~/.bashrc #Start Panda (Run from terminal)
.././qemu-system-i386 -hda OS/wheezy_panda2.qcow2 -m 3500 -net nic -net user -monitor stdio

QEMU 2.8.50 monitor - type 'help' for more information
(qemu) 

## Runing Asidstory
* Run block "Collect System Call Data" 
    * Change for the correct `./Recordings/<recording_name>` 
    * Change the output filename `<gitPull_raw.txt>`

In [178]:
%%bash
. ~/.bashrc #Collect Pandalog Data
.././qemu-system-i386 -m 3500 -replay ./Recordings/gitPull -panda asidstory -os linux-32-lava32

os_type=2 bits=32 os_details=[lava32]
panda_require: osi
Looking for kconffile in /opt/panda3/panda/i386-softmmu/Debian7/.././panda/plugins/osi_linux/kernelinfo.conf
OSI grabbing Linux introspection backend.
Linux OSI, using group lava32:32 from /opt/panda3/panda/i386-softmmu/panda/plugins/osi_linux/kernelinfo.conf.
panda_require: osi_linux
loading snapshot
... done.
opening nondet log for read :	./Recordings/gitPull-rr-nondet.log
max_instr = 455574782
./Recordings/gitPull-rr-nondet.log:   455574782 instrs total.
gitPull:     4614315 (  1.01%) instrs.    2.03 sec.  0.20 GB ram.
gitPull:     9140961 (  2.01%) instrs.    3.05 sec.  0.21 GB ram.
gitPull:    13673790 (  3.00%) instrs.    4.28 sec.  0.21 GB ram.
gitPull:    18307630 (  4.02%) instrs.    5.73 sec.  0.22 GB ram.
gitPull:    22793970 (  5.00%) instrs.    7.03 sec.  0.22 GB ram.
gitPull:    27473083 (  6.03%) instrs.    8.52 sec.  0.22 GB ram.
gitPull:    31936548 (  7.01%) instrs.    9.85 sec.  0.22 GB ram.
gitPull:    3648212

Initializing plugin asidstory
Initializing plugin osi
Initializing plugin osi_linux
INFO(/opt/panda3/panda/panda/plugins/osi_linux/osi_linux.cpp:init_plugin): Read kernel info from group "lava32:32" of file "/opt/panda3/panda/i386-softmmu/panda/plugins/osi_linux/kernelinfo.conf".
INFO(/opt/panda3/panda/panda/plugins/osi_linux/osi_linux.cpp:init_plugin): osi_linux initialization complete.


## Running Syscall2
* Run block "Collect System Call Data" 
    * Change for the correct `./Recordings/<recording_name>` 
    * Change the output filename `Syscall_Data/<gitPull_raw.txt>`

In [179]:
%%bash
. ~/.bashrc #Collect System Call Data 
.././qemu-system-i386 -m 3500 -replay ./Recordings/gitPull -panda syscalls2:profile=linux_x86 -os linux-32-lava32 \
    > Syscall_Data/gitPull_raw1.txt

Initializing plugin syscalls2


## Parsing Syscall2 Data
* Run the following three blocks of code
    * Imports
    * 

In [26]:
#Imports 
import glob #used for file IO
import csv #used to write csv file\n

In [180]:
#Parse file to readable .csv file
raw = "gitPull_raw1.txt" #this should match the filename in 
out = "gitPull1.csv"
rawFile = open("Syscall_Data/"+raw)
csvFile = open("Syscall_Data/"+out, 'w')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['Program Counter', 'Panda Current ASID', 'System Number',
                    'Syscall Return Type','System Call Name']) #header
setupSyscall()
for line in rawFile:
    words =  line.split(" ")
    if(words[0] == "Syscall:"):
        csvWriter.writerow( [words[1].strip(), words[2].strip(), words[3].strip(),
                             syscalls[int(words[3])][0], syscalls[int(words[3])][1]])
rawFile.close()
csvFile.close()

In [150]:
#Returns more information on the System Call
def setupSyscall():
    syscalltype = open("../../panda/plugins/syscalls2/linux_x86_prototypes.txt")
    syscalls = {}
    for line in syscalltype:
        words = line.split(" ")
        syscalls[int(words[0])] = [words[1]," ".join(words[2:]).strip()]
